# 라이브러리 설치 및 불러오기

In [ ]:
# Selenium, BeautifulSoup4, Requests 설치
!pip install selenium # 동적 페이지 크롤링 (무한스크롤)
!pip install beautifulsoup4 # HTML 파싱
!pip install requests

# apt 업데이트
!apt-get update

# chromedriver 설치
!apt install chromium-chromedriver

# 해당 경로에 설치한 chromedriver 복사
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:4 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:5 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:7 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:10 https://developer.download.nvidia.com/comp

# 크롤링

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import csv
import requests
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import re


def crawl_infinite_scroll_website_and_save_to_csv(urls, names, csv_filename):
    # 웹 드라이버 초기화
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    names_len = len(names)

    # CSV 파일을 쓰기 모드로 열기
    with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(["category", "question", "answer"]) # 제목행

        for i in range(names_len):
        # for i in range(1):
          site_name = names[i]
          url = urls[site_name]
          print('==========', site_name, ' : ', url, '사이트 크롤링 진행중', '==========')

          # FAQ 웹사이트에 접속
          driver = webdriver.Chrome(options=chrome_options)
          driver.get(url)

          # 해당 사이트의 Question들 추출
          items = driver.find_elements(By.CSS_SELECTOR, 'div.lb-peek-expander')  # 원하는 요소의 클래스명
          print(len(items))

          # 데이터를 CSV 파일에 작성
          # 모든 Question에 대해서 Answer 추출 (Questions 순회)
          for item in items:
              category=''; question=''; answer='';

              # BeautifulSoup을 사용하여 HTML 파싱
              soup = BeautifulSoup(item.get_attribute('innerHTML'), 'html.parser')

              # Question 추출
              soup.find('div', _class="lb-expand-trigger")
              question = soup.select_one('.lb-txt').get_text()
              question = question.replace(">>", "").replace("»", "").replace("Q:", "").strip()
              print(question)

              # Answer 추출
              soup.find('div', _class="lb-content")
              temp = soup.select('.lb-content-inner .lb-rtxt p')
              if len(temp) == 1 :
                answers = temp[0:]
              else :
                answers = temp[1:] # 1번째 p태그는 질문임 => 질문을 제외한 나머지만
              # answers = ["<p>" + answer.get_text() + "</p>" for answer in answers]
              answers = [answer.get_text() for answer in answers]
              answer = ' '.join(answers).strip()
              print(answer)

              print(f'"category : {site_name}, question : {question}, answer : {answer}"')
              csv_writer.writerow([site_name, question, answer])

          # 브라우저 닫기
          driver.quit()

if __name__ == "__main__":
    # 크롤링할 웹사이트 이름에 따른 URL 딕셔너리
    website_urls = {
        'API Gateway': 'https://aws.amazon.com/ko/api-gateway/faqs/',
        'Elastic Beanstalk' : 'https://aws.amazon.com/ko/elasticbeanstalk/faqs/'
    }

    # 크롤링할 웹사이트 이름 배열
    website_name = ['API Gateway', 'Elastic Beanstalk']

    # 저장할 CSV 파일명 설정
    csv_filename = "crawling.csv"

    # 함수 호출하여 크롤링 및 CSV 저장 수행
    crawl_infinite_scroll_website_and_save_to_csv(website_urls, website_name, csv_filename)


========== API Gateway  :  https://aws.amazon.com/ko/api-gateway/faqs/ 사이트 크롤링 진행중 ==========
66
Amazon API Gateway란 무엇입니까?
Amazon API Gateway는 어떤 규모에서든 개발자가 API를 손쉽게 게시, 유지 관리, 모니터링 및 보호할 수 있도록 지원하는 완전관리형 서비스입니다. AWS Management Console에서 몇 번의 클릭으로 애플리케이션이 백엔드 서비스(Amazon Elastic Compute Cloud(Amazon EC2), Amazon Elastic Container Service(Amazon ECS) 또는 AWS Elastic Beanstalk에서 실행되는 애플리케이션, AWS Lambda에서 실행되는 코드, 기타 웹 애플리케이션 등)의 데이터, 비즈니스 로직 또는 기능에 액세스할 수 있도록 "현관문" 역할을 하는 API를 생성할 수 있습니다. Amazon API Gateway는 트래픽 관리, 권한 부여 및 액세스 제어, 모니터링, API 버전 관리를 비롯해 최대 수십만 건의 동시 API 호출을 수락 및 처리하는 데 관련된 모든 작업을 처리합니다. Amazon API Gateway에는 최소 요금이나 시작 비용이 없습니다. HTTP API와 REST API의 경우 수신한 API 호출과 전송한 데이터 양에 대해서만 요금을 지불하면 됩니다. WebSocket API의 경우 전송하고 수신한 메시지와 사용자/디바이스가 WebSocket API에 연결한 시간에 대해서만 요금을 지불하면 됩니다.
"category : API Gateway, question : Amazon API Gateway란 무엇입니까?, answer : Amazon API Gateway는 어떤 규모에서든 개발자가 API를 손쉽게 게시, 유지 관리, 모니터링 및 보호할 수 있도록 지원하는 완전관리형 서비스입니다. AWS Management Console에서 몇 번의 클릭으로 애플리케